<a href="https://colab.research.google.com/github/devyadav11/DL_assignment_2/blob/main/Dl_assignment_2_Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.utils import shuffle # for shuffling
import os
import cv2
import random
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import gc



In [2]:
#!pip install wandb
import wandb
# !wandb login
wandb.login(key="500727e3b54d5202c75cd3136d6326f29e89e05f")

#key = 500727e3b54d5202c75cd3136d6326f29e89e05f

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ee23m074 (devyadav11) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip
!rm nature_12K.zip

--2025-04-20 16:35:07--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   184MB/s    in 27s     

2025-04-20 16:35:35 (134 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [5]:
#import model

model = models.resnet50(weights='ResNet50_Weights.DEFAULT')



Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 159MB/s]


In [6]:
model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

for prem in model.parameters():
    prem.requires_grad = False

# change number of number of neurons in last layers
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [7]:
## check which parameters are frozen and active
for name, prem in model.named_parameters():
    print(prem, prem.requires_grad)

Streaming output truncated to the last 5000 lines.
         [[ 0.0380]]],


        ...,


        [[[ 0.0766]],

         [[ 0.0815]],

         [[-0.0272]],

         ...,

         [[-0.0078]],

         [[ 0.0350]],

         [[-0.0237]]],


        [[[ 0.0121]],

         [[-0.0073]],

         [[-0.0072]],

         ...,

         [[-0.0066]],

         [[-0.0436]],

         [[-0.0483]]],


        [[[-0.0221]],

         [[-0.0010]],

         [[ 0.0519]],

         ...,

         [[ 0.0202]],

         [[-0.0093]],

         [[-0.0354]]]]) False
Parameter containing:
tensor([3.2165, 2.0340, 2.1040, 3.6064, 1.8920, 2.6308, 2.4051, 2.5590, 2.4148,
        2.4717, 2.1721, 1.9654, 2.9141, 2.2997, 2.0517, 3.2925, 2.1472, 1.8631,
        3.1182, 2.2752, 2.4814, 2.5940, 1.8417, 3.0514, 2.9708, 2.5209, 2.2752,
        2.7768, 2.8290, 2.7711, 1.9431, 2.2267, 2.2885, 2.5403, 2.2340, 3.0228,
        3.1027, 4.1674, 3.2425, 2.5349, 2.3253, 2.6113, 2.7270, 2.0020, 2.3322,
        1.5061, 3

In [8]:
print(model.named_parameters)

<bound method Module.named_parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d

In [9]:
image_hight = 256
image_width = 256

In [10]:
##load dataset

def load_data(batchSize):
  transform_data = transforms.Compose([ transforms.Resize((image_hight, image_width)), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ])   # normalise = output = (input - mean) / std ) (convert to (-1 to 1)

  train_data = datasets.ImageFolder(root = "/content/inaturalist_12K/train", transform=transform_data)

  class_index = train_data.class_to_idx

  training_idx = []
  val_idx  = []

  for train_class, train_index in class_index.items():
    index = []
    for k, (img_path, lable) in enumerate(train_data.samples):
      if lable == train_index:
        index.append(k)


    train_indices, val_indices = train_test_split(index, test_size=0.2, random_state=42)
    training_idx.extend(train_indices)           # extend creates a single flat list
    val_idx.extend(val_indices)

  training_data = Subset(train_data, training_idx)
  val_data = Subset(train_data, val_idx)




  train_loader = DataLoader(training_data, batch_size=batchSize, shuffle=True, pin_memory= True )
  val_loader = DataLoader(val_data, batch_size=batchSize, shuffle=True, pin_memory= True)

  test_data = datasets.ImageFolder(root = "/content/inaturalist_12K/val", transform=transform_data)
  test_laoder = DataLoader(test_data, batch_size=batchSize, shuffle=True, pin_memory= True)



  return train_loader, val_loader, test_laoder


In [11]:
def find_accuracy(model, criterion, dataLoader, dataName):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f'{dataName} Loss: {val_loss/len(dataLoader)},'
          f'{dataName} Accuracy: {100*correct/total}%\n')

    #wandb.log({f'{dataName}_accuracy': 100*correct/total})
    #wandb.log({f'{dataName}_loss': val_loss/len(dataLoader)})

In [12]:
def pretrained_modal_training(optimiser_fn, num_epochs, batch_size,
                   activation_fxn, batch_norm, dropout_prob, weight_decay, learning_rate):





  train_loader, val_loader, test_loader = load_data(batch_size)

  if activation_fxn == "relu":
    activation_fxn = nn.ReLU()
  elif activation_fxn == "leaky_relu":
    activation_fxn = nn.LeakyReLU()
  elif activation_fxn == "elu":
    activation_fxn = nn.ELU()
  else:
    activation_fxn = nn.Sigmoid()



  model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

  for prem in model.parameters():
      prem.requires_grad = False

# change number of number of neurons in last layers
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 10)




  criterion = nn.CrossEntropyLoss()

  if optimiser_fn == "adam":
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimiser_fn == "nadam":
    optimizer = optim.NAdam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimiser_fn == "rmsprop":
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
 # stocastic gradient decent
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)




  model = torch.nn.DataParallel(model,device_ids = [0]).to(device)

  for epoch in range(num_epochs):
    model.train()

    for index, (inputs, labels) in enumerate(tqdm(train_loader, desc=f'Training Progress {epoch+1}')):
      inputs, labels = inputs.to(device), labels.to(device)                        # keeps input and lables run on same GPU, or CPU if u choose multiple devices
      optimizer.zero_grad()                                                        # set gradients to zero
      outputs = model(inputs)                                                      # forward pass
      loss = criterion(outputs, labels)                                            # calculate loss
      loss.backward()                                                              # backward pass
      optimizer.step()                                                             # updates the parameters after back prop


    find_accuracy(model, criterion, train_loader, "train")
    find_accuracy(model, criterion, val_loader, "validation")


  return model


In [13]:
modal = pretrained_modal_training(optimiser_fn = "nadam", num_epochs = 10, batch_size =64,
                   activation_fxn = "elu", batch_norm = True, dropout_prob = 0.1, weight_decay = 0.005, learning_rate = 1e-4)



Training Progress 1: 100%|██████████| 125/125 [01:41<00:00,  1.23it/s]


train Loss: 1.9576942977905274,train Accuracy: 54.644330541317665%

validation Loss: 1.9650556817650795,validation Accuracy: 55.25%



Training Progress 2: 100%|██████████| 125/125 [01:30<00:00,  1.38it/s]


train Loss: 1.6937115039825439,train Accuracy: 63.39542442805351%

validation Loss: 1.705530535429716,validation Accuracy: 62.9%



Training Progress 3: 100%|██████████| 125/125 [01:30<00:00,  1.38it/s]


...

train Loss: 1.067807963848114,train Accuracy: 75.07188398549819%

validation Loss: 1.1115182526409626,validation Accuracy: 73.7%



Training Progress 9: 100%|██████████| 125/125 [01:31<00:00,  1.37it/s]


train Loss: 1.0146333003044128,train Accuracy: 76.08451056382047%

validation Loss: 1.0698110777884722,validation Accuracy: 73.9%



Training Progress 10: 100%|██████████| 125/125 [01:31<00:00,  1.36it/s]


train Loss: 0.9738923640251159,train Accuracy: 77.04713089136142%

validation Loss: 1.0257483292371035,validation Accuracy: 74.7%

